In [1]:
import sqlite3
con = sqlite3.connect("kqlquery.db")

In [3]:
sql_create_db = """
CREATE TABLE FieldEntity ( 
	field                VARCHAR(100) NOT NULL    ,
	entity               VARCHAR(100) NOT NULL    ,
	CONSTRAINT pk_FieldEntity PRIMARY KEY ( field, entity ),
	CONSTRAINT unq_FieldEntity_field UNIQUE ( field )
 );

CREATE TABLE KqlQuery ( 
	source_path          VARCHAR(1000) NOT NULL    ,
	query                TEXT(10000)     ,
	name                 VARCHAR(100)     ,
	query_id             INTEGER NOT NULL  PRIMARY KEY  ,
	local_path           VARCHAR(1000) NOT NULL    
 );

CREATE TABLE QueryAttribute ( 
	query_id             INTEGER NOT NULL    ,
	attribute_name       VARCHAR(100) NOT NULL    ,
	attribute_value      VARCHAR(1000)     ,
	CONSTRAINT pk_QueryAttribute PRIMARY KEY ( query_id, attribute_name ),
	FOREIGN KEY ( query_id ) REFERENCES KqlQuery( query_id )  
 );

CREATE TABLE QueryField ( 
	query_id             INTEGER NOT NULL    ,
	field                VARCHAR(100) NOT NULL    ,
	CONSTRAINT pk_QueryField PRIMARY KEY ( query_id, field ),
	CONSTRAINT unq_QueryField_field UNIQUE ( field ),
	FOREIGN KEY ( query_id ) REFERENCES KqlQuery( query_id )  ,
	FOREIGN KEY ( field ) REFERENCES FieldEntity( field )  
 );

CREATE TABLE QueryFunction ( 
	query_id             INTEGER NOT NULL    ,
	function             VARCHAR(100) NOT NULL    ,
	CONSTRAINT pk_QueryFunction PRIMARY KEY ( query_id, function ),
	FOREIGN KEY ( query_id ) REFERENCES KqlQuery( query_id )  
 );

CREATE TABLE QueryOperator ( 
	query_id             INTEGER NOT NULL    ,
	operator             VARCHAR(100) NOT NULL    ,
	CONSTRAINT pk_QueryOperator PRIMARY KEY ( query_id, operator ),
	CONSTRAINT unq_QueryOperator_query_id UNIQUE ( query_id ),
	FOREIGN KEY ( query_id ) REFERENCES KqlQuery( query_id )  
 );

CREATE TABLE QueryTable ( 
	table_name           VARCHAR(100) NOT NULL    ,
	query_id             INTEGER NOT NULL    ,
	CONSTRAINT pk_QueryTable PRIMARY KEY ( query_id, table_name ),
	CONSTRAINT unq_QueryTable_table_name UNIQUE ( table_name ),
	FOREIGN KEY ( query_id ) REFERENCES KqlQuery( query_id )  
 );

CREATE TABLE OperatorFieldReference ( 
	query_id             INTEGER NOT NULL    ,
	field                VARCHAR(100) NOT NULL    ,
	operator             VARCHAR(100) NOT NULL    ,
	CONSTRAINT pk_OperatorFieldReference PRIMARY KEY ( query_id, operator, field ),
	FOREIGN KEY ( query_id ) REFERENCES QueryOperator( query_id )  ,
	FOREIGN KEY ( field ) REFERENCES QueryField( field )  
 );

CREATE TABLE OperatorTableReference ( 
	query_id             INTEGER NOT NULL    ,
	operator             VARCHAR(100) NOT NULL    ,
	table_name           VARCHAR(100) NOT NULL    ,
	CONSTRAINT pk_OperatorTableReference PRIMARY KEY ( query_id, operator, table_name ),
	FOREIGN KEY ( query_id ) REFERENCES QueryOperator( query_id )  ,
	FOREIGN KEY ( table_name ) REFERENCES QueryTable( table_name )  
 );

CREATE VIEW KqlQueryField AS SELECT * FROM QueryFile
INNER JOIN KqlQuery ON QueryFile.query_id == KqlQuery.query_id;

CREATE VIEW KqlQueryTable AS SELECT * FROM QueryTable
INNER JOIN KqlQuery ON QueryTable.query_id = KqlQuery.query_id;

CREATE VIEW dbs_validate_view AS SELECT * FROM QueryFile
INNER JOIN KqlQuery ON QueryFile.query_id == KqlQuery.query_id;
"""

In [5]:
cur = con.cursor()
# for statement in sql_create_db.split(";"):
#     cur.execute(statement)

cur.executescript(sql_create_db)

In [1]:
from sqlalchemy import create_engine
db_engine = create_engine('sqlite:///kqlquery.db')

In [3]:
from db_schema import *

In [4]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = db_engine)
session = Session()

In [5]:
new_query = KqlQuery(
    source_path="/home/ian/query2",
    local_path = "/home/ian/query2.json",
    name = "Query2",
)

C:\Users\Ian\AppData\Local\Temp\ipykernel_162452\2502972332.py:1: SAWarning: relationship 'QueryField.query' will copy column KqlQuery.query_id to column QueryField.query_id, which conflicts with relationship(s): 'FieldEntity.querys' (copies KqlQuery.query_id to QueryField.query_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="querys"' to the 'QueryField.query' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  new_query = KqlQuery(
C:\Users\Ian\AppData\Local\Temp\ipykernel_162452\2502972332.py:1: SAWarning: relationship 'QueryField.entity' will copy column FieldEntity.field to column QueryField.field,

In [11]:
session.add(new_query)
session.commit()

In [14]:
t_tables = [QueryTable(table_name=name) for name in ["one", "two"]]


In [2]:
session

NameError: name 'session' is not defined

In [17]:


# KqlQuery.tables = relationship("QueryTable", order_by = QueryTable.table_name, back_populates = "queries")
# QueryTable.queries = relationship("KqlQuery", back_populates = "tables")

C:\Users\Ian\AppData\Local\Temp\ipykernel_648048\1118387722.py:1: SAWarning: Property KqlQuery.tables on mapped class KqlQuery->KqlQuery being replaced with new property KqlQuery.tables; the old property will be discarded
  KqlQuery.tables = relationship("QueryTable", order_by = QueryTable.table_name, back_populates = "queries")
C:\Users\Ian\AppData\Local\Temp\ipykernel_648048\1118387722.py:1: SAWarning: relationship 'KqlQuery.tables' will copy column KqlQuery.query_id to column QueryTable.query_id, which conflicts with relationship(s): 'QueryTable.query' (copies KqlQuery.query_id to QueryTable.query_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the

In [2]:
url = "https://learn.microsoft.com/en-us/azure/azure-monitor/reference/tables/officeactivity"
import pandas as pd
pd.read_html(url)[0]

,Column,Type,Description
0,AADGroupId,string,Azure Active Directory group id
1,AADTarget,string,The user that the action (identified by the Op...
2,Actor,string,The user or service principal that performed t...
3,ActorContextId,string,The GUID of the organization that the actor be...
4,ActorIpAddress,string,The actor's IP address in IPV4 or IPV6 address...
...,...,...,...
123,UserDomain,string,The domain of the user
124,UserId,string,The UPN (User Principal Name) of the user who ...
125,UserKey,string,An alternative ID for the user identified in t...
126,UserSharedWith,string,The user that a resource was shared with


In [ ]:
schema_cats_url = "https://learn.microsoft.com/en-us/azure/azure-monitor/reference/tables/tables-category"
import requests

resp = requests.get(schema_cats_url)

from bs4 import BeautifulSoup
BeautifulSoup

result

In [41]:
dir(result)
def get_security_category_list(resp):
    soup = BeautifulSoup(resp.text, "html.parser")

    result = soup.find("div", class_="content")
    sec_header =result.find("h2", id="security")
    return sec_header.find_next_sibling()

In [42]:
def build_table_index(security_cat_list):
    table_prefix = "https://learn.microsoft.com/azure/azure-monitor/reference/tables/{href}"
    return {
        item.a.contents[0]: {
            "href": item.a.attrs.get("href"),
            "url": table_prefix.format(**(item.a.attrs)),
        }
        for item in security_cat_list.find_all("li")
    }

sec_cat_list = get_security_category_list(resp)
sec_url_dict = build_table_index(sec_cat_list)


In [45]:
type(resp)

requests.models.Response

In [34]:
from tqdm.auto import tqdm
import asyncio

def read_table_from_url(table, ref):
    table_data = pd.read_html(ref["url"])[0]
    columns = table_data.columns
    table_data["Table"] = table
    table_data["Url"] = ref["url"]
    return table_data.set_index("Table")

def fetch_table_schemas(sec_url_dict):
    print("reading schema")
    all_tables = [
        read_table_from_url(table, ref)
        for table, ref in tqdm(sec_url_dict.items())
    ]
    return pd.concat[all_tables]

comb_tables = fetch_table_schemas(sec_url_dict)

100%|██████████| 98/98 [00:14<00:00,  6.67it/s]


,Column,Type,Description,table,url
0,AppCorrelationID,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
1,AttributeLDAPDisplayName,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
2,AttributeSyntaxOID,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
3,AttributeValue,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
4,Category,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
5,CorrelationId,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
6,DSName,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
7,DSType,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
8,NewObjectDN,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...
9,ObjectClass,string,NaN,AADDomainServicesDirectoryServiceAccess,https://learn.microsoft.com/azure/azure-monito...


In [37]:
test_tab = all_tables[10].copy()
test_tab.set_index("table")

,Column,Type,Description,url
table,,,,
AADRiskyServicePrincipals,AccountEnabled,bool,true if the service principal account is enabl...,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,AppId,string,The globally unique identifier for the associa...,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,CorrelationId,string,The ID for correlated log analytics events. Ca...,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,DisplayName,string,The display name for the service principal.,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,Id,string,The unique identifier assigned to the service ...,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,IsProcessing,bool,Indicates whether Azure AD is currently proces...,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,OperationName,string,Name of the operation.,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,RiskDetail,string,Details of the detected risk.,https://learn.microsoft.com/azure/azure-monito...
AADRiskyServicePrincipals,RiskLastUpdatedDateTime,datetime,The date and time that the risk state was last...,https://learn.microsoft.com/azure/azure-monito...


In [72]:
df = pd.concat(all_tables)
df["hash"] = pd.util.hash_pandas_object(df["Column"])
display(df)
pd.util.hash_pandas_object(pd.DataFrame([{"Column": "AADTenantId"}])).at[0]

,Column,Type,Description,table,url,Column 1,Column 2,Column 3,...,Column 498,hash
0,AADTenantId,string,The AAD tenant ID of associated request.,AADB2CRequestLogs,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,14048459883259893479
1,CallerIpAddress,string,Hashed format of caller IP address. Can be use...,AADB2CRequestLogs,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,9428068023137835893
2,CorrelationId,string,The ID for correlated log analytics events. Ca...,AADB2CRequestLogs,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,2781777946879618025
3,OperationName,string,Name of the operation.,AADB2CRequestLogs,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,10395961821423712288
4,_ResourceId,string,A unique identifier for the resource that the ...,AADB2CRequestLogs,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,7349900439462902064
...,...,...,...,...,...,...,...,...,...,...,...
41,_SubscriptionId,string,A unique identifier for the subscription that ...,WireData,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,15352049308617374827
42,TimeGenerated,datetime,Time of the record,WireData,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,6623577775155001585
43,TLPLevel,string,Traffic Light Protocol (TLP) Level is one of t...,WireData,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,17795497464082604254
44,TotalBytes,long,Total number of bytes sent during session,WireData,https://learn.microsoft.com/azure/azure-monito...,NaN,NaN,NaN,NaN,NaN,1338932357246339281


14048459883259893479